## 9장 나이브 베이즈

### (1) 가우시안 나이브 베이즈 실습

- 변수 지정 및 전처리

In [1]:
#1. 모듈 및 함수 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

#2. 데이터 불러오기
df = pd.read_csv('Ashopping.csv', encoding = 'cp949')

#3. 변수 선택(독립변수/종속변수)
X = df[['구매유형', '거래기간', '구매카테고리수']]
Y = df['할인민감여부']

#4. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#5. 표준화 및 원핫인코딩
ct = ColumnTransformer([('scaling', StandardScaler(), ['거래기간', '구매카테고리수']), ('onehot', OneHotEncoder(sparse = False),[ '구매유형'])]).fit(X_train)
X_train=ct.transform(X_train)
X_test=ct.transform(X_test)



Using TensorFlow backend.


- 모형 학습 및 예측

In [2]:
#1. 모듈 및 함수 임포트
from sklearn.naive_bayes import GaussianNB

#2. 모형 생성
model= GaussianNB()

#3. 모형 학습 및 예측
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

print('평가용 데이터 세트에 대한 예측값\n', Y_pred)

평가용 데이터 세트에 대한 예측값
 [0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0
 1 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0
 1 0 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 1 1 0 1 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1
 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0
 0 0 0 0]


- 모형 평가

In [3]:
print('학습용 데이터 세트 정확도: {:.3f}'.format(model.score(X_train, Y_train)))
print('평가용 데이터 세트 정확도: {:.3f}'.format(model.score(X_test, Y_test)))

학습용 데이터 세트 정확도: 0.713
평가용 데이터 세트 정확도: 0.710


In [4]:
from sklearn.metrics import classification_report  

Y_pred=model.predict(X_test)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.72      0.82      0.77       177
           1       0.68      0.54      0.61       123

    accuracy                           0.71       300
   macro avg       0.70      0.68      0.69       300
weighted avg       0.71      0.71      0.70       300



- 분류 확률의 추정치 확인

In [5]:
yproba = model.predict_proba(X_test)
yproba[0:9].round(2)

array([[0.79, 0.21],
       [0.74, 0.26],
       [0.58, 0.42],
       [0.85, 0.15],
       [0.7 , 0.3 ],
       [0.5 , 0.5 ],
       [0.93, 0.07],
       [0.62, 0.38],
       [0.43, 0.57]])

### (2) 베르누이 나이브 베이즈 실습

- 데이터 살펴보기

In [6]:
#1. 모듈 및 함수 불러오기
import pandas as pd

#2. 데이터 불러오기
data = pd.read_csv('spam.csv', encoding='latin1')

#3. 상위 5개 행 불러오기
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1) 
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data.head()


,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
print('메일의 개수: %d' % len(data))
print('레이블 값 분포:\n', pd.Series(data['v1'].value_counts()))


메일의 개수: 5572
레이블 값 분포:
 0    4825
1     747
Name: v1, dtype: int64


- 데이터 전처리

In [9]:
#1. 모듈 및 함수 불러오기
from sklearn.feature_extraction.text import CountVectorizer

#2. 변수 지정(독립변수/종속변수)
X= data['v2']
Y= data['v1']

#3. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#4. 문서 단어 행렬 (DTM)으로 변환
cv=CountVectorizer(binary=True, stop_words='english', min_df=3)
X_train= cv.fit_transform(X_train)
X_test= cv.transform(X_test)

#5. 변환 결과 출력
print('단어별 인덱스 부여 결과: \n', cv.vocabulary_)
print(' ')
print('문서 단어 행렬 변환 결과: \n', X_train.toarray())

단어별 인덱스 부여 결과: 
 {'ok': 1222, 'care': 356, 'umma': 1792, 'don': 561, 'make': 1065, 'life': 997, 'time': 1729, 'laugh': 970, 'add': 144, 'years': 1960, 'surely': 1658, 'ur': 1810, 'gud': 765, 'ni8': 1189, 'swt': 1666, 'dreams': 568, 'great': 758, 'new': 1187, 'offer': 1218, 'double': 564, 'mins': 1115, 'txt': 1781, 'best': 265, 'orange': 1239, 'tariffs': 1674, 'latest': 968, 'camera': 349, 'phones': 1275, 'free': 691, 'mobileupd8': 1132, '08000839402': 9, '2stoptxt': 71, 'cs': 479, 'second': 1483, 'wish': 1909, 'wonderful': 1923, 'll': 1013, 'sch': 1473, 'fr': 689, 'dun': 580, 'haf': 773, 'da': 490, 'book': 288, 'home': 829, 'pa': 1250, 'lunch': 1052, 'aha': 160, 'finally': 666, 'fancy': 642, 'chat': 383, 'flirt': 676, 'sexy': 1509, 'yr': 1969, 'area': 200, 'just': 933, 'reply': 1416, 'summer': 1647, 'join': 920, 'stop': 1620, 'walk': 1859, 'cut': 487, 'road': 1436, 'right': 1431, 'lt': 1049, 'gt': 763, 'street': 1626, 'haha': 774, 'got': 753, 'fast': 647, 'lose': 1033, 'weight': 1892, 

- 모형 학습 및 예측

In [10]:
#1. 모듈 및 함수 불러오기
from sklearn.naive_bayes import BernoulliNB 

#2. 모형 학습 및 예측
model = BernoulliNB()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test) 
print('평가용 데이터 세트에 대한 예측값\n', Y_pred)

평가용 데이터 세트에 대한 예측값
 [0 0 0 ... 0 0 0]


- 모형 평가

In [11]:
print('학습용 데이터 세트 정확도: {:.3f}'.format(model.score(X_train, Y_train)))
print('평가용 데이터 세트 정확도: {:.3f}'.format(model.score(X_test, Y_test)))

학습용 데이터 세트 정확도: 0.988
평가용 데이터 세트 정확도: 0.976


In [12]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      1434
           1       0.99      0.84      0.91       238

    accuracy                           0.98      1672
   macro avg       0.98      0.92      0.95      1672
weighted avg       0.98      0.98      0.98      1672



### (3) 다항분포 나이브 베이즈 실습

- 데이터 살펴보기

In [13]:
#1. 모듈 및 함수 불러오기
from sklearn.datasets import fetch_20newsgroups

#2. 데이터 불러오기
newsgroups=fetch_20newsgroups(subset='all')

#3. 데이터 속성 확인하기
dir(newsgroups)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [15]:
print('카테고리 리스트: %s' % newsgroups.target_names )
print(' ')
print('-------------첫 번째 뉴스 정보-------------')
print(' ')
print('카테고리: %s' % newsgroups.target_names[newsgroups.target[0]])
print(' ')
print(newsgroups.data[0])


카테고리 리스트: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
 
-------------첫 번째 뉴스 정보-------------
 
카테고리: rec.sport.hockey
 
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I th

- 변수 지정 및 데이터 전처리

In [16]:
#1. 모듈 및 함수 불러오기
from sklearn.feature_extraction.text import TfidfVectorizer

#2. 데이터 불러오기
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups= fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), categories=categories) 

#3. 변수 지정(독립변수/종속변수)
X = newsgroups.data
Y = newsgroups.target

#4. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#5. 문서 단어 행렬 변환
tv = TfidfVectorizer()
X_train = tv.fit_transform(X_train)
X_test = tv.transform(X_test)

#6. 변환 결과 출력
print('단어별 인덱스 부여 결과: \n', tv.vocabulary_)
print(' ')
print('문서 단어 행렬 변환 결과: \n', X_train.toarray())


단어별 인덱스 부여 결과: 
 {'can': 5727, 'certainly': 6055, 'see': 23410, 'opposing': 18933, 'the': 26027, 'amen': 3379, 'but': 5579, 'that': 26025, 'doesn': 9252, 'require': 22259, 'moment': 17545, 'of': 18752, 'silence': 23876, 'does': 9251, 'anybody': 3646, 'else': 9865, 'besides': 4819, 'me': 16885, 'vicious': 27833, 'circle': 6377, 'here': 13007, 'guarantee': 12541, 'you': 28948, 'people': 19690, 'who': 28405, 'want': 28148, 'school': 23203, 'prayer': 20521, 'aren': 3865, 'going': 12245, 'to': 26353, 'back': 4423, 'off': 18755, 'when': 28369, 'they': 26113, 'even': 10432, 'manage': 16540, 'get': 12062, 'quiet': 21361, 'for': 11421, 'their': 26039, 'kids': 15276, 'pray': 20519, 'silently': 23880, 'this': 26149, 'isn': 14694, 'true': 26795, 'many': 16616, 'are': 3861, 'forced': 11428, 'use': 27450, 'automobile': 4309, 'system': 25611, 'don': 9288, 'it': 14737, 'by': 5605, 'choice': 6285, 'if': 13627, 'there': 26088, 'were': 28326, 'other': 19117, 'ways': 28238, 'getting': 12069, 'around': 392

- 모형 학습 및 예측

In [17]:
#1. 모듈 및 함수 불러오기
from sklearn.naive_bayes import MultinomialNB 

#2. 모형 학습 및 예측
model = MultinomialNB()
model.fit(X_train, Y_train)
Y_pred= model.predict(X_test)
Y_pred

array([0, 0, 2, ..., 1, 1, 0], dtype=int64)

- 모형 평가

In [18]:
print('학습용 데이터 세트 정확도: {:.3f}'.format(model.score(X_train, Y_train)))
print('평가용 데이터 세트 정확도: {:.3f}'.format(model.score(X_test, Y_test)))


학습용 데이터 세트 정확도: 0.850
평가용 데이터 세트 정확도: 0.733


In [19]:
from sklearn import metrics
print('정밀도의 평균: {:.3f}'.format(metrics.precision_score(Y_test, Y_pred, average='macro')))
print('재현율의 평균: {:.3f}'.format(metrics.recall_score(Y_test, Y_pred, average='macro')))
print('F1 스코어의 평균: {:.3f}' .format(metrics.f1_score(Y_test, Y_pred, average='macro')))


정밀도의 평균: 0.763
재현율의 평균: 0.681
F1 스코어의 평균: 0.639
